# Tugas 1 - Crawling Data (detik.com)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://food.detik.com/indeks"]
categories = ["Kuliner"]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    for page in range(1, 4):  # Looping untuk beralih halaman
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            break

        url = f"{base_url}/{page}"
        get_data(url, category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman

# Membuat dataframe dari list data
df = pd.DataFrame({
    "judul": judul,
    "tanggal": tanggal,
    "isi": isi
})

# Menyimpan dataframe ke file CSV
df.to_csv("Crawl-berita.csv", index=False)

In [4]:
df=pd.read_csv("Crawl-berita.csv")
df

,judul,tanggal,isi
0,Legit Kenyal! 20 Mochi Daifuku Ada di Sini,"Senin, 02 Sep 2024 14:00 WIB",Jakarta - Mochi daifuku asal Jepang kini sedan...
1,Arummi Foods Jadi Juara 1 di Food Startup Indo...,"Senin, 02 Sep 2024 13:15 WIB",Jakarta - Arummi Foods menjadi juara 1 dalam a...
2,Rasis! Pria Berkulit Hitam Disebut 'Monyet' sa...,"Senin, 02 Sep 2024 13:00 WIB",Jakarta - Seorang pria berkulit hitam mengalam...
3,Tak Sabar! Pria Ini Mancing Dapat Ikan Langsun...,"Senin, 02 Sep 2024 12:30 WIB","Jakarta - Definisi 'fast food', pria ini meman..."
4,Itutuh: Pho dengan Iga Sapi dan Banh Mi Rasa A...,"Senin, 02 Sep 2024 12:00 WIB",Jakarta -\n \n Di Tebet ada rest...
5,Gading Marten Jualan Nasi Tempong Lauk Ayam Go...,"Senin, 02 Sep 2024 11:30 WIB",Jakarta - Gading Marten punya restoran sambal ...
6,"Resep Tumis Kangkung yang Gurih dan Lezat, Coc...","Senin, 02 Sep 2024 11:00 WIB",Tentang\nBahan\nLangkah\n\nJakarta - Tumis kan...
7,6 Air Rebusan Daun yang Bantu Mengobati Asam U...,"Senin, 02 Sep 2024 10:30 WIB",Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\t6 Rebusan ...
8,Resep Pesmol Ikan Mas Berbumbu Rempah yang Sedap,"Senin, 02 Sep 2024 10:00 WIB",Tentang\nBahan\nLangkah\n\nJakarta - Ikan mas ...
9,Jus Mangga Tanpa Buah hingga Rekomendasi Kulin...,"Senin, 02 Sep 2024 09:30 WIB",Jakarta -\n \n Pembuatan jus man...
